In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)
print( df.shape )
df.head()

(163955180, 5)


,session,aid,ts,type,day
0,0,1517085,1659304800,0,0
1,0,1563459,1659304904,0,0
2,0,1309446,1659367439,0,0
3,0,16246,1659367719,0,0
4,0,1781822,1659367871,0,0


In [3]:
AIDS = df.aid.unique().values
DAYS = df.day.unique().values

In [4]:
DAYS

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [5]:
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']

In [6]:
tmp.head()

,,item,user
aid,day,,
1098752,1,1,1
896347,5,8,8
1344221,5,1,1
184988,0,5,5
299712,1,3,3


In [7]:
import cupy
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,21).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')

In [8]:
dff = dff.set_index( ['aid','day'] )

In [9]:
dff.head()

Empty DataFrame
Columns: []
Index: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]

In [10]:
dff = dff.merge(tmp, left_index=True, right_index=True, how='left').fillna(0).reset_index()

In [11]:
dff.head()

,aid,day,item,user
0,354,4,0,0
1,354,5,0,0
2,354,6,0,0
3,354,7,0,0
4,354,8,0,0


In [12]:
item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

In [13]:
del dff
gc.collect()

53

In [14]:
item_features3.head()

,item_day_std,user_day_std
aid,,
382021,1.532194,1.532194
926129,1.364516,1.364516
157066,1.414214,1.414214
949601,0.478091,0.478091
717823,6.776008,6.776008


In [15]:
df['mn'] = df.groupby('session').ts.transform('min')

In [16]:
df = df.loc[df.mn > mn + 60*60*24*3]
df.shape

(89752927, 6)

In [17]:
df['t'] = df.ts - df.mn

In [18]:
df = df.loc[df.t < 60*60*24*7]
df.shape

(73820535, 7)

In [19]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique'})
item_features.columns = ['buy_ratio','count_item','count_user']

In [20]:
item_features.head()

,buy_ratio,count_item,count_user
aid,,,
0,0.000000,14,12
1,0.052632,19,17
2,0.000000,7,7
3,0.079235,732,392
4,0.000000,45,37


In [21]:
item_features2 = df.groupby('aid').agg({'type':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std']

f32 = ['buy_ratio_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [22]:
item_features2.head()

,buy_ratio_std
aid,
324514,0.277350
1677842,0.707107
117139,0.267261
1101880,0.000000
1164251,0.000000


In [23]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [24]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [25]:
item_features.columns = [f'{c}_5' for c in item_features.columns]

In [26]:
item_features.head()

,buy_ratio_5,count_item_5,count_user_5,repeat_5,buy_ratio_std_5,item_day_std_5,user_day_std_5
aid,,,,,,,
0,0.000000,14,12,1.166667,0.000000,1.660465,1.660465
1,0.052632,19,17,1.117647,0.229416,2.657424,2.657424
2,0.000000,7,7,1.000000,0.000000,0.925820,0.925820
3,0.079235,732,392,1.867347,0.312539,82.133194,82.133194
4,0.000000,45,37,1.216216,0.000000,3.540245,3.540245


In [27]:
item_features.dtypes

buy_ratio_5        float32
count_item_5         int32
count_user_5         int32
repeat_5           float32
buy_ratio_std_5    float32
item_day_std_5     float32
user_day_std_5     float32
dtype: object

In [28]:
item_features.to_parquet('../../data/item_user_features/item12.pqt')